In [2]:
from zhipuai_llm import ZhipuAILLM

In [3]:
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [4]:
zhipuai_model = ZhipuAILLM(model="chatglm_std", temperature=0.1, api_key=api_key)
zhipuai_model("你好，请你自我介绍一下！")

D:\tools\developmentSoftwores\anaconda3\envs\PrimacyEffectGPT-main\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' 你好，我是 智谱清言，是清华大学KEG实验室和智谱AI公司共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。'

In [5]:
import sys
sys.path.append("../C2 使用 LLM API 开发应用") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

In [8]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：40


In [9]:
question = "什么是prompt engineering?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


In [10]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


检索到的第0个内容: 
 相反，我们应通过 Prompt 指引语言模型进行深入思考。可以要求其先列出对问题的各种看法，说明推理依据，然后再得出最终结论。在 Prompt 中添加逐步推理的要求，能让语言模型投入更多时间逻辑思维，输出结果也将更可靠准确。

综上所述，给予语言模型充足的推理时间，是 Prompt Engineering 中一个非常重要的设计原则。这将大大提高语言模型处理复杂问题的效果，也是构建高质量 Prompt 的关键之处。开发者应注意给模型留出思考空间，以发挥语言模型的最大潜力。

2.1 指定完成任务所需的步骤

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。

首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成英语。第三，在英语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：英语摘要和人名个数。要求输出以换行符分隔。
-----------------------------------------------------
检索到的第1个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型准确理解我们的意图，就像向一个外星人详细解释人类世界一样。过于简略的 Prompt 往往使模型难以把握所要完成的具体任务。

其次，让语言模型有充足时间推理也极为关键。就像人类解题一样，匆忙得出的结论多有失误。因此 Prompt 应加入逐步推理的要求，给模型留出充分思考时间，这样生成的结果才更准确可靠。

如果 Prompt 在这两点上都作了优化，语言模型就能够尽可能发挥潜力，完成复杂的推理和生成任务。掌握这些 Prompt 设计原则，是开发者取得语言模型应用成功的重要一步。

一、原则一 编写清晰、具体的指令
----------------------------------------------

In [16]:


client = ZhipuAILLM()
client("你好")



'你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。'

In [13]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(client,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [21]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
南瓜书是对周志华教授的《机器学习》（西瓜书）中部分公式推导细节进行补充和解析的辅助学习材料，旨在帮助读者更好地理解机器学习中的数学推导，它是由一群数学基础扎实的爱好者编写的笔记形式的学习资料。南瓜书建议与西瓜书结合使用，遇到难以理解的部分再查阅。谢谢你的提问！


In [22]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
王阳明是明代著名的哲学家、军事家、教育家，心学的代表人物，提出了“知行合一”的理论。他的哲学思想对后世有着深远的影响，尤其在东亚文化圈内有很高的地位。谢谢你的提问！


In [23]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
client.predict(prompt_template)

'南瓜书是对周志华教授所著的经典机器学习教材《机器学习》（俗称“西瓜书”）中涉及的公式进行详细解析和推导的辅助教材。它由开源组织Datawhale发起，团队成员针对西瓜书中较为难以理解的公式进行了梳理和解释，特别是对那些跳跃性较大、推导过程复杂的公式进行了详细的步骤解析。\n\n南瓜书的名字寓意着它是西瓜书的有益补充，就像咖啡伴侣一样，旨在帮助读者更好地理解和掌握机器学习中的数学基础和推导过程。书中不仅对西瓜书中的250个重难点公式进行了覆盖率达99%的解析和推导，还补充了必要的先验知识，使得即使数学基础不是特别扎实的读者也能够跟上并理解复杂的数学推导。\n\n南瓜书自2019年发布以来，因其内容实用、讲解清晰，受到了广大读者的喜爱和好评，并在GitHub上获得了很高的关注度，star数量突破了1.6万。此外，南瓜书还得到了业界的认可，多位人工智能领域的大咖对其进行了推荐。后来，应读者要求，南瓜书以纸质书的形式出版，让更多的学习者能够方便地使用这一学习资源。'

In [24]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
client.predict(prompt_template)

'王阳明，即王守仁（1472-1529年），是明代著名的哲学家、军事家、教育家和文化名人。他在中国哲学史上占有极其重要的地位，是心学的代表人物之一，其学说被后人称为“阳明心学”。王阳明不仅学问渊博，在军事和政治上也有卓越的成就，他曾成功平定宁王叛乱，在江西等地为官时提倡廉洁政治，改善民生，因此深受百姓爱戴。他的“知行合一”理论强调内心认知和实践的统一，主张通过修身齐家治国平天下的方式，实现个人的道德修养和社会的整体和谐。王阳明的思想对后世产生了深远的影响，被后人尊为“千古完人”。'

In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

In [26]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    client,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

是的，你可以通过本模块的学习掌握关于提示工程的知识，包括提示词设计的最佳实践和技巧，以及如何通过优化提示词来提升大语言模型应用的性能。内容涵盖提示原则、软件开发提示词设计、文本总结、推理、转换、扩展和构建聊天机器人等方面。


In [27]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

需要教授关于提示工程的知识，是因为对于开发者来说，掌握如何设计清晰、具体且能充分发挥大语言模型潜力的提示词（Prompt）是至关重要的。这不仅能帮助开发者通过API接口高效地构建软件应用程序，还能提升大语言模型在各种应用场景中的性能和效果。通过学习提示工程，开发者可以更好地理解如何提供充足的上下文，明确表达需求，以及如何给予模型充分的思考时间来进行推理和生成任务，从而使得语言模型能够更准确、更可靠地完成复杂的任务，激发开发者的想象力，创造出更出色的语言模型应用。


In [ ]:
import streamlit as st